In [1]:
from torchvision import datasets

train_dataset = datasets.cifar.CIFAR100(root='cifar100', train=True, transform=None, download=False)
test_dataset = datasets.cifar.CIFAR100(root='cifar100', train=False, transform=None, download=False)

In [2]:
import collections
import math
import os
import shutil
import pandas as pd
import torch
import torchvision
from torch import nn
from d2l import torch as d2l
from torch.nn import functional as F
import numpy as np
from torchvision import transforms
import cv2
import glob
import random

In [3]:
d2l.DATA_HUB['cifar10_tiny'] = (d2l.DATA_URL + 'kaggle_cifar10_tiny.zip',
                                '2068874e4b9a9f0fb07ebe0ad2b29754449ccacd')

demo = False

if demo:
    data_dir = d2l.download_extract('cifar10_tiny')
else:
    data_dir = '/Users/lizijian/data/cifar-100-python/cutout_dataset'

In [4]:
labels = dict(((str(i),str(train_dataset.targets[i])) for i in range(50000)))
print('# 训练样本 :', len(labels))
print('# 类别 :', len(set(labels.values())))

# 训练样本 : 50000
# 类别 : 100


In [5]:
class Cutout(object):
    """Randomly mask out one or more patches from an image.
    Args:
        n_holes (int): Number of patches to cut out of each image.
        length (int): The length (in pixels) of each square patch.
    """
    def __init__(self, n_holes=1, length=16):
        self.n_holes = n_holes
        self.length = length
 
    def __call__(self, img):
        """
        Args:
            img (Tensor): Tensor image of size (C, H, W).
        Returns:
            Tensor: Image with n_holes of dimension length x length cut out of it.
        """
        h = img.size(1)
        w = img.size(2)
 
        mask = np.ones((h, w), np.float32)
 
        for n in range(self.n_holes):
            y = np.random.randint(h)
            x = np.random.randint(w)
 
            y1 = np.clip(y - self.length // 2, 0, h)
            y2 = np.clip(y + self.length // 2, 0, h)
            x1 = np.clip(x - self.length // 2, 0, w)
            x2 = np.clip(x + self.length // 2, 0, w)
 
            mask[y1: y2, x1: x2] = 0.
 
        mask = torch.from_numpy(mask)
        mask = mask.expand_as(img)
        img = img * mask
 
        return img
# 执行cutout
def cutout(png,cutout_png):
    img = cv2.imread(png)
    img = transforms.ToTensor()(img)
    cut = Cutout(length=8)
    img = cut(img)

    # cutout图像写入本地
    img = img.mul(255).byte()
    img = img.numpy().transpose((1, 2, 0))
    cv2.imwrite(cutout_png, img)

In [6]:
#用cutout处理训练数据集
path = '/Users/lizijian/data/cifar-100-python/cutout_dataset/train/*'
a = 0
for i in random.sample(range(0,50000),10000):
    a+=1
    file_abs = glob.glob(path)[i]
    cutout(file_abs,file_abs)
    print(a,file_abs)

1 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/13477_18+caterpillar_s_001004.png
2 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/42411_29+edmontonia_s_000024.png
3 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/44702_48+minibike_s_001249.png
4 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/7815_54+orchid_s_001543.png
5 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/12597_85+armored_combat_vehicle_s_000031.png
6 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/8087_31+indian_elephant_s_001122.png
7 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/35923_55+otter_s_002244.png
8 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/48464_65+lapin_s_001837.png
9 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/35183_13+school_bus_s_000572.png
10 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/7008_58+pickup_s_001757.png
11 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/12

93 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/45271_41+mower_s_000844.png
94 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/3552_16+beer_can_s_001447.png
95 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/47097_78+serpent_s_000063.png
96 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/21146_46+abel_s_000083.png
97 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/2097_81+streetcar_s_001539.png
98 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/31131_88+panthera_tigris_s_000469.png
99 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/19206_34+fox_s_001501.png
100 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/11613_12+drawbridge_s_002345.png
101 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/6683_89+tractor_s_001585.png
102 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/29571_62+poppy_s_002358.png
103 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/25823_27+croco

181 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/19058_37+house_s_000413.png
182 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/35488_14+butterfly_s_002991.png
183 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/9522_87+surveillance_system_s_000918.png
184 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/1919_82+sunflower_s_000404.png
185 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/36545_6+honeybee_s_001114.png
186 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/33685_59+pine_tree_s_002078.png
187 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/37521_82+sunflower_s_000005.png
188 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/45105_51+mushroom_s_001360.png
189 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/34847_54+orchid_s_001599.png
190 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/28901_37+bungalow_s_000258.png
191 /Users/lizijian/data/cifar-100-python/cutout_data

269 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/14997_25+couch_s_001610.png
270 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/13978_2+infant_s_000819.png
271 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/9555_47+sugar_maple_s_000313.png
272 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/16349_41+lawn_mower_s_000979.png
273 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/9836_72+seal_s_001899.png
274 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/43434_8+bicycle_s_002410.png
275 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/42967_9+bottle_s_001535.png
276 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/39727_46+adam_s_000647.png
277 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/47201_11+boy_s_001336.png
278 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/268_66+raccoon_s_001641.png
279 /Users/lizijian/data/cifar-100-python/cutout_dataset/train/5661_2+baby_s_000332.

KeyboardInterrupt: 